In [ ]:
import boto3
import json
import base64

In [ ]:
sagemaker = boto3.client('sagemaker')

# Ensure to modify the model name, model url and role arn to match your sagemaker instance
model_name = "MODEL-NAME"
model_url = "https://<INSERT-YOUR-BUCKET-NAME>.s3.amazonaws.com/<INSERT-YOUR-TAR-NAME>.tar.gz"  # S3 URL to your model artifact - Found in S3
role_arn = "arn:aws:iam::<INSERT-AWS-ID>:role/service-role/<APPROPRIATE-SAGEMAKER-ROLE>"  # Your SageMaker IAM role ARN found in the Sagemaker Dashboard

# instantiate the deep learning container hosting both Hugging Face transformers and PyTorch - you can choose a model here
# https://github.com/aws/deep-learning-containers/blob/master/available_images.md
container = {
    'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:2.1.0-transformers4.37.0-cpu-py310-ubuntu22.04',  
    'ModelDataUrl': model_url
}

create_model_response = sagemaker.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role_arn,
    PrimaryContainer=container
)

print(create_model_response)

In [ ]:
endpoint_config_name = "ENDPOINT-CONFIG-NAME"

create_endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',  # Choose the instance type based on your needs
            'InitialVariantWeight': 1
        },
    ]
)

# This will be used within lambda to access your endpoint accordingly
endpoint_name = "ENDPOINT-NAME"
create_endpoint_response = sagemaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print(create_endpoint_response)

In [ ]:
# Opening a local jpg file - must be stored in JupyterLab instance located on the lefthand side of the console
with open("<INSERT-TEST-IMAGE-NAME>.jpg", "rb") as f:
    # reading as bytes
    image = f.read()

# Content type to be sent with API call
content_type = 'image/jpg'

In [ ]:
# Test Endpoint - This can be tested before accessing the endpoint from anywhere else

sagemaker_runtime = boto3.client('sagemaker-runtime')

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=image
)

In [ ]:
# Decode and print the response
result = json.loads(response['Body'].read().decode())
print(result)